In [11]:
!pip install pyqt5
!pip install webdriver_manager
!pip install PyQtWebEngine

In [ ]:
import math, shutil, sys, os, csv, time, pprint, json, csv, requests
import urllib.request
import pandas as pd  
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5.QtWebEngineWidgets import QWebEnginePage
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Pa

In [ ]:
def get_keys(dict): 
    list = [] 
    for key in dict.keys(): 
        list.append(key)       
    return list

In [ ]:
def create_list(d, keys):
    keys_d = get_keys(d)

    if set(keys_d) == set(keys):
        return list(d.values())
    else:
        dictionary = {}
        for key in keys:
            dictionary[key] = d[key] if key in d else None
        return list(dictionary.values())

In [ ]:
categories = {'uncategorized': [1, 21654], 'vision': [5,5825], 'autograd': [7,3036], 'nlp': [8,1282], 'c': [11,1073], 'distributed': [12,560], 'jit': [13,374], 'reinforcement-learning': [6,333], 'deployment': [14,227], 'quantization': [17,203], 'mobile': [18,124], 'audio': [9,109], 'glow': [10,68], 'site-feedback': [3,64], 'ignite': [15,34], 'memory-format': [23,23], 'mixed-precision': [27,15], 'complex': [22,14], 'windows': [26,13], 'captum': [21,10], 'xla': [25,7], 'hackathon': [16,6], 'jobs': [24,2], 'tensorboard': [28,0]}
# cats = get_keys(categories)
# categories[cats[0]]
# categories['tensorboard'][1]

In [ ]:
# FETCH TITLES AND META-DATA
cats = get_keys(categories)
topics_per_page = 30

for cat in cats:
    topics_info = []
    keys = []
    num_pages = categories[cat][1]/topics_per_page
    for i in range(0, math.ceil(num_pages)): 
        time.sleep(0.5)
        my_url = 'https://discuss.pytorch.org/c/' + cat + '/' + str(categories[cat][0]) + '.json?page='+ str(i)
        # my_url

        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        print(my_url+' fetched...')

        encoding = 'utf-8'
        filename = '/'+cat+'/'+cat+'_'+str(i)+'.json'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        f = open(filename, "w")
        f.write(page_html.decode(encoding))
        f.close()
        print(filename+ ' file created')

        with open(filename) as json_file: 
            lst = []    
            dict = json.load(json_file) 
            # str(dict['users'])
            # str(dict['topic_list'])
            topic_list = dict['topic_list']
            topics = topic_list['topics'] # list of dictionary
            print('topics have been fetched from '+ filename)

            if i == 0:
                keys = get_keys(topics[0])
            
            for d in topics:
                row = create_list(d, keys)
                topics_info.append(row)
            print('Values/Rows added for ' + str(i) + '-th page')

    # print(topics_info)
    df = pd.DataFrame(topics_info)
    df.columns = keys
    csv_data = df.to_csv(index=False)
    df.to_csv('topics_info_'+cat+'.csv', index=False)
    print('CSV file created. End of '+cat+ ' category\n\n')

    time.sleep(10)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for cat in cats:
    shutil.move("/content/topics_info_"+cat+".csv", "/content/drive/My Drive/AREA_51.2/Research/PyTorch Discussion Forum Data/Titles")

In [21]:
# Fetch Description if all titles

keys = ['id', 'title', 'created_at', 'reply_count', 'views', 'description', 'creator_link', 'creator_name', 'creator_alias', 'post_date', 'post_likes', 'replies', 'repliers_links', 'reply_dates', 'reply_likes']
cats = get_keys(categories) # commented for debugging 
# cats = ['site-feedback'] # added for debugging 

for cat in cats:
    csv_list = []
    filename = "/content/drive/My Drive/AREA_51.2/Research/PyTorch Discussion Forum Data/Titles/topics_info_"+cat+".csv"
    if categories[cat][1] != 0:
        data = pd.read_csv(filename)
        for i in range(0, data.shape[0]):
            time.sleep(0.5)
            temp_lst = []
            topic_url = "https://discuss.pytorch.org/t/"+data.at[i,'slug']+"/"+str(data.at[i,'id'])

            while True:
                try:
                    uClient = uReq(topic_url)
                    page_html = uClient.read()
                    if uClient.getcode() == 200:
                        uClient.close()
                        break
                except Exception as inst:
                    print(inst)
            
            page_soup = soup(page_html, "html.parser")
            
            temp_lst.append(data.at[i,'id'])
            temp_lst.append(data.at[i,'fancy_title'])
            temp_lst.append(data.at[i,'created_at'])
            temp_lst.append(data.at[i,'reply_count'])
            temp_lst.append(data.at[i,'views'])

            all_posts = page_soup.findAll("div",{"class":"post"}) # commented for debugging
            temp_lst.append(str(all_posts[0]).replace("\n", "<NewLine>")) # description # commented for debugging

            all_creators = page_soup.findAll("span",{"class":"creator"})
            temp_lst.append(all_creators[0].a['href']) # creator_link
            temp_lst.append(" ".join(all_creators[0].text.split()[1:])) # creator_name
            temp_lst.append(all_creators[0].span.text) # creator_alias

            all_reply_dates = page_soup.findAll("span",{"class":"crawler-post-infos"})
            temp_lst.append(all_reply_dates[0].time.text.strip()) # post_date

            all_reply_likes = page_soup.findAll("span",{"class":"post-likes"})
            temp_lst.append(all_reply_likes[0].text) # post_like

            replies = ""
            for p in range(1, len(all_posts)):
                replies += "REPLY "+str(p)+": "+ str(all_posts[p]).replace("\n", "<NewLine>")+"; <NewLine> "
            temp_lst.append(replies)

            repliers_links = ""
            for c in range(1, len(all_creators)):
                repliers_links += "REPLIER "+str(c)+": "+all_creators[c].a['href']+"; <NewLine> "
            temp_lst.append(repliers_links)

            reply_dates = ""
            for d in range(1, len(all_reply_dates)):
                reply_dates += "REPLY_DATE "+str(d)+": "+all_reply_dates[d].time.text.strip()+"; <NewLine> "
            temp_lst.append(reply_dates)

            reply_likes = ""
            for l in range(1, len(all_reply_likes)):
                reply_likes += "REPLY "+str(l)+" LIKES: "+all_reply_likes[l].text+"; <NewLine> "
            temp_lst.append(reply_likes)        
            
            # print(description, "\n\n", replies, "\n\n", creator_link, creator_alias, creator_name, "\n\n", repliers_links, "\n\n", post_date, "\n\n", reply_dates, "\n\n", post_like, "\n\n", reply_likes)
            
            csv_list.append(temp_lst)
            if i%50 == 0:
                print("Topic Details for # "+str(i)+" from Category "+cat+" has been added to csv_list\t"+str(data.shape[0]-i)+" number of topics left\t Estimated Remaining time for this category: "+str(int(((data.shape[0]-i)*0.6)/60)) + " minutes")
    
    if categories[cat][1] != 0:
        df = pd.DataFrame(csv_list)
        df.columns = keys
        df.to_csv('topics_details_'+cat+'.csv', index=False)
        print('\nCSV file created. End of '+cat+ ' category\n\n')
    else:
        print('No Topic exists in '+cat+' category')
    time.sleep(10)

Topic Details for # 0 from Category c has been added to csv_list	1078 number of topics left	 Estimated Remaining time for this category: 10 minutes
Topic Details for # 50 from Category c has been added to csv_list	1028 number of topics left	 Estimated Remaining time for this category: 10 minutes
Topic Details for # 100 from Category c has been added to csv_list	978 number of topics left	 Estimated Remaining time for this category: 9 minutes
Topic Details for # 150 from Category c has been added to csv_list	928 number of topics left	 Estimated Remaining time for this category: 9 minutes
Topic Details for # 200 from Category c has been added to csv_list	878 number of topics left	 Estimated Remaining time for this category: 8 minutes
Topic Details for # 250 from Category c has been added to csv_list	828 number of topics left	 Estimated Remaining time for this category: 8 minutes
Topic Details for # 300 from Category c has been added to csv_list	778 number of topics left	 Estimated Remaini

In [ ]:
# Run this in the console to prevent Google Colab from disconnecting
function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,3600000)